# 🎸 Guitar Strum Generator - Dataset Exploration

**Notebook 01: Explore Available Datasets**

This notebook explores the Chordonomicon dataset and other resources to understand what's available for training our guitar chord/strum generator.

---

**Key Finding:** No public dataset provides symbolic strumming patterns in "D_DU_UD_" format.  
**Our Strategy:** Combine synthetic generation (70%) with real progressions from Chordonomicon (30%).

---

## 1. Setup

In [ ]:
# Install required packages
!pip install -q datasets pandas matplotlib seaborn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

print("✅ Setup complete!")

## 2. Load Chordonomicon

Chordonomicon contains **680,000 chord progressions** extracted from real songs.  
Source: https://huggingface.co/datasets/ailsntua/Chordonomicon

In [ ]:
from datasets import load_dataset

print("📥 Loading Chordonomicon from Hugging Face...")
print("   (First time may take several minutes)\n")

dataset = load_dataset("ailsntua/Chordonomicon", split="train")
df = dataset.to_pandas()

print(f"✅ Loaded {len(df):,} entries!")
print(f"\n📋 Columns available:")
for col in df.columns:
    non_null = df[col].notna().sum()
    print(f"   - {col}: {non_null:,} non-null values")

In [ ]:
# Display first few rows
df.head(3)

## 3. Understand the Chord Format

Chordonomicon uses **structural tags** in the chord strings:
- `<intro_1>`, `<verse_1>`, `<chorus_1>`, etc.
- These indicate song sections

In [ ]:
# Look at a few chord progression examples
print("🎵 Sample chord progressions:\n")

for i in range(5):
    row = df.iloc[i]
    chords = str(row['chords'])
    # Truncate for display
    if len(chords) > 150:
        chords = chords[:150] + "..."
    
    print(f"Example {i+1}:")
    print(f"  {chords}")
    print()

In [ ]:
# Extract and count section tags
def extract_sections(chord_string):
    """Extract section tags from chord string."""
    pattern = r'<(\w+)_?\d*>'
    return re.findall(pattern, str(chord_string))

# Count all sections across dataset
all_sections = []
for chords in df['chords'].dropna().sample(10000):  # Sample for speed
    all_sections.extend(extract_sections(chords))

section_counts = Counter(all_sections)

print("📊 Most common section tags:")
for section, count in section_counts.most_common(10):
    print(f"   <{section}>: {count:,}")

## 4. Extract Unique Chords

Let's see what chord symbols appear in the dataset.

In [ ]:
def extract_chords(chord_string):
    """Extract chord symbols from chord string."""
    # Remove section tags
    clean = re.sub(r'<\w+_?\d*>', ' ', str(chord_string))
    # Split and filter
    tokens = clean.split()
    # Basic chord pattern
    chord_pattern = r'^[A-Ga-g][#b]?(m|maj|min|dim|aug|sus|add|7|9|11|13|M)*\d*$'
    chords = [t for t in tokens if re.match(chord_pattern, t, re.IGNORECASE)]
    return chords

# Extract chords from sample
all_chords = []
for chord_str in df['chords'].dropna().sample(50000):
    all_chords.extend(extract_chords(chord_str))

chord_counts = Counter(all_chords)

print(f"📊 Found {len(chord_counts):,} unique chord symbols")
print(f"\nMost common chords:")
for chord, count in chord_counts.most_common(20):
    pct = count / len(all_chords) * 100
    bar = '█' * int(pct)
    print(f"   {chord:8} {count:>6,} ({pct:4.1f}%) {bar}")

## 5. Genre Distribution

In [ ]:
# Main genre distribution
if 'main_genre' in df.columns:
    genre_counts = df['main_genre'].value_counts()
    
    plt.figure(figsize=(12, 6))
    genre_counts.head(15).plot(kind='barh', color='steelblue')
    plt.xlabel('Number of Songs')
    plt.title('Top 15 Genres in Chordonomicon')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Genre statistics:")
    for genre in genre_counts.head(10).index:
        count = genre_counts[genre]
        pct = count / len(df) * 100
        print(f"   {genre:20} {count:>8,} ({pct:5.1f}%)")

## 6. Decade Distribution

In [ ]:
if 'decade' in df.columns:
    decade_counts = df['decade'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 5))
    decade_counts.plot(kind='bar', color='coral')
    plt.xlabel('Decade')
    plt.ylabel('Number of Songs')
    plt.title('Songs by Decade in Chordonomicon')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 7. Chord Progression Length Analysis

In [ ]:
# Calculate progression lengths
def count_chords(chord_string):
    return len(extract_chords(chord_string))

# Sample for speed
sample_df = df.sample(10000)
lengths = sample_df['chords'].apply(count_chords)

print(f"📊 Chord progression length statistics (sample of 10,000):")
print(f"   Min: {lengths.min()}")
print(f"   Max: {lengths.max()}")
print(f"   Mean: {lengths.mean():.1f}")
print(f"   Median: {lengths.median():.0f}")

plt.figure(figsize=(12, 5))
plt.hist(lengths[lengths < 200], bins=50, color='purple', alpha=0.7)
plt.xlabel('Number of Chords')
plt.ylabel('Frequency')
plt.title('Distribution of Chord Progression Lengths')
plt.axvline(lengths.mean(), color='red', linestyle='--', label=f'Mean: {lengths.mean():.1f}')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Key Finding: What's Missing?

### ❌ What Chordonomicon DOES NOT have:
- **Strumming patterns** (D_DU_UD_ format)
- **Natural language descriptions/prompts**
- **Tempo information**
- **Emotion labels**

### ✅ What Chordonomicon DOES have:
- **Real chord progressions** from 680,000 songs
- **Genre labels** (rock, pop, folk, etc.)
- **Structural tags** (verse, chorus, etc.)
- **Decade information**

### 💡 Our Strategy:
1. **Extract real chord progressions** from Chordonomicon
2. **Add strumming patterns** using our rule-based system
3. **Generate natural language prompts** using templates
4. **Combine with synthetic samples** for complete coverage

In [ ]:
# Summary
print("="*60)
print("📊 CHORDONOMICON SUMMARY")
print("="*60)
print(f"\n✅ Total entries: {len(df):,}")
print(f"✅ Has genre labels: {'main_genre' in df.columns}")
print(f"✅ Has structural tags: Yes (verse, chorus, etc.)")
print(f"✅ Real chord progressions: Yes")
print(f"\n❌ Strumming patterns: NO - must add synthetically")
print(f"❌ Natural language prompts: NO - must generate")
print(f"❌ Tempo information: NO - must infer/assign")
print(f"\n💡 Use case for thesis:")
print(f"   Sample ~75 real progressions as 30% of training data")
print(f"   Add strumming + prompts using our rule-based system")
print("="*60)

## 9. Next Steps

Continue to **Notebook 02: Build Dataset** to create the training data!